In [26]:
#   bem: triangulation and fmm/bem electrostatics tools
#
#   Copyright (C) 2011-2012 Robert Jordens <jordens@gmail.com>
#
#   This program is free software: you can redistribute it and/or modify
#   it under the terms of the GNU General Public License as published by
#   the Free Software Foundation, either version 3 of the License, or
#   (at your option) any later version.
#
#   This program is distributed in the hope that it will be useful,
#   but WITHOUT ANY WARRANTY; without even the implied warranty of
#   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#   GNU General Public License for more details.
#
#   You should have received a copy of the GNU General Public License
#   along with this program.  If not, see <http://www.gnu.org/licenses/>.

# `bem` 2D Surface Trap example
SimpleTrap

In [27]:
import sys
import logging, os
from time import time
import numpy as np
# Importing pyface.qt is for setting "Qt" sip API version to 2 before matplotlib sets it to default v1
# which is incompatible with "pyside" in package "mayavi". Python 2.
# import pyface.qt
import matplotlib.pyplot as plt
from multiprocessing import Pool

sys.path.append('/Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/ionLifetimes/bem')
sys.path.append('/Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/ionLifetimes/bem/examples')
sys.path.append('/Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/ionLifetimes/electrode')

from bem import Electrodes, Sphere, Mesh, Grid, Configuration, Result
from bem.formats import stl
from trap_library import *
import numpy as np

In [23]:
# base file name for outputs and inputs is the script name
try:
    # works only if we are a script
    prefix = os.path.splitext(__file__)[0]
    print("non-used")
except NameError:
    # fallback for notebooks
    stl_file_in = "htrapf"

In [24]:
# scale to natural units (ion height)
scale = 40e-6    # Distance from ion to electrode is 40 um.
use_stl = True

if not use_stl:
    # load electrode faces from loops
    ele = Electrodes.from_trap(open("%s.ele" % prefix), scale)
    # initial triangulation, area 20, quiet
    mesh = Mesh.from_electrodes(ele)
    mesh.triangulate(opts="qa10Q")
else:
    # load electrode faces from colored stl
    # s_nta is intermediate processed stl file.
    s_nta = stl.read_stl(open("trapstl/%s.stl" % stl_file_in, "rb"))
    print("Import stl:",os.path.abspath("./"+prefix+".stl"),"\n")
    print("Electrode colors (numbers):\n")
    mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=scale / 1e-3, rename={0: "DC21"}))

Import stl: /home/sqip/Documents/github/bem/examples/SimpleTrap/htrap.stl 

Electrode colors (numbers):

dropping 12319
dropping 31754
dropping 1055
dropping 3171
dropping 8223
dropping 10271
dropping 6175
dropping 13343
dropping 31
dropping 7199
dropping 30653
dropping 17439
dropping 2079
dropping 1023
dropping 14367
dropping 568
dropping 18463
dropping 24607
dropping 23583
dropping 21535
dropping 8776
dropping 15391
dropping 20511
dropping 4127


The formal rename of electrode. Assign each electrode a string name instead of its color coding. Use the numbers you get above.
stl.stl_to_mesh() prints normal vectors (different faces) in each electrode.

In [25]:
print(len(s_nta), type(s_nta),"\n")
# s_nta is a length 3 tuple. (normal, triangle, attribute)
# Normal direction of each triangle, three vetice of triangles, coding number of colors.
print("Triangles:",len(s_nta[0]),"\nColors:",len(s_nta[2]),"\n")    # This isn't right.

# stl_to_mesh() only assigns names and does scaling, doing no triangulation to stl mesh.
# "scale=scale/1e-6" only scales dimensionless scale/1e-6.    1e-6: if stl uses micron as unit.

mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=1,
    rename=el_colordict[stl_file_in], quiet=False))

3 <class 'tuple'> 

Triangles: 1968 
Colors: 1968 

dropping 12319
1 planes in electrode DC1
normals vectors:
 [[-2.60208521e-16  0.00000000e+00  1.00000000e+00]]
1 planes in electrode DC11
normals vectors:
 [[4.15363807e-19 8.95639721e-16 1.00000000e+00]]
1 planes in electrode DC12
normals vectors:
 [[4.15363807e-19 5.11564066e-16 1.00000000e+00]]
1 planes in electrode DC13
normals vectors:
 [[4.15363807e-19 2.55442214e-16 1.00000000e+00]]
1 planes in electrode DC14
normals vectors:
 [[ 4.15363807e-19 -3.26298366e-17  1.00000000e+00]]
1 planes in electrode DC15
normals vectors:
 [[ 4.15363807e-19 -3.84755320e-16  1.00000000e+00]]
1 planes in electrode DC16
normals vectors:
 [[ 4.15363807e-19 -6.72827387e-16  1.00000000e+00]]
1 planes in electrode DC17
normals vectors:
 [[ 4.15363807e-19 -8.64895813e-16  1.00000000e+00]]
1 planes in electrode DC18
normals vectors:
 [[ 4.15363807e-19 -1.05696424e-15  1.00000000e+00]]
1 planes in electrode DC21
normals vectors:
 [[8.00688916e-16 0.000000

In [ ]:
# ### Generate triangle mesh with constraints
#
# The meshes are 2-dimensional triangles on the surface of electrodes. The region enclosed by constraint shape can have finer mesh. Triangulation is done by `triangle` C library.
#there are all in units of mm now (Ben S. feb 2022)
xl = (3.1)*72*1e-3
yl = -0.051*72*1e-3
zl = 1.06*72*1e-3
rad = 5*72*1e-3
size = 100.0
# set .1 max area within 3
# areas_from_constraints specifies sphere with finer mesh inside it.
mpl.rcParams['lines.linewidth'] = 0.2
 # "inside", "outside" set different mesh densities.
# mesh.areas_from_constraints(Sphere(center=np.array([xl,yl,zl]),
#            radius=10*factor, inside=0.1*factor**2, outside=1000))
# mesh.triangulate(opts="",new = False)
mesh.areas_from_constraints(Sphere(center=np.array([xl,yl,zl]),
           radius=rad, inside=2e-4, outside=2e-3))
# retriangulate quality and quiet with areas
mesh.triangulate(opts="qQ", new=False)
# save base mesh to vtk
mesh.to_vtk(prefix)
# grid to evalute potential and fields at


n, s = 100, 0.002
Lx, Ly, Lz = 0.100,0.100,0.100 # in the unit of scaled length l
sx, sy, sz = s, s, s

vtk_out = "vtks/htrapf"
print("done")

# ni is grid point number, si is step size. Thus to fix size on i direction you need to fix ni*si.
nx, ny, nz = [2*np.ceil(L/2.0/s).astype('int') for L in (Lx, Ly, Lz)]
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(xl,yl,zl), step=(sx, sy, sz), shape=(nx, ny, nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("Grid origin/l:", grid.get_origin())

n, s = 2*10, .1
grid = Grid(center=(0, 0, 1.5), step=(s, s, s), shape=(n, n, n))
# generate electrode potential configurations to simulate
# use regexps to match electrode names
jobs = list(Configuration.select(mesh, "DC.*", "RF"))
# run the different electrodes on the parallel pool
#pmap = Pool().map # parallel map
pmap = map # serial map
t0 = time()
list(pmap(run_job, ((job, grid, prefix) for job in jobs)))  # In python 3, convert map(...) to list(map(...))
print("Computing time: %f s"%(time()-t0))

In [ ]:
# isocontour plot of the RF pseudopotential radially
result = Result.from_vtk(prefix, "RF")
p = result.pseudo_potential
x = grid.to_mgrid()[:, p.shape[0]//2]    # In python 3, use //
p = p[p.shape[0]//2]
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.contour(x[1], x[2], p, levels=np.linspace(0, 2e-2, 20), cmap=plt.cm.Reds)

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(aspect="equal"))
mesh.plot(ax)

In [ ]:
# explore it in fancy 3D
# fire up a mayavi2 window showing base mesh, charges on final mesh
# and isosurfaces of the pseudopotential
Result.view(prefix, "RF")
# need to start the full eventloop for the window.
# close it to return control to the notebook
from pyface.api import GUI
GUI().start_event_loop()
"done"

In [ ]:
from electrode import System, GridElectrode

# load the electrostatics results into a electrode.System()
s = System()
for name in "DC1 DC2 DC3 DC4 DC5 RF".split():
    r = Result.from_vtk(prefix, name)
    e = GridElectrode.from_result(r)
    e.name = name
    s.append(e)

In [ ]:
import scipy.constants as ct
l = 40e-6 # length scale
o = 100e6*2*np.pi # rf frequency
m = 25*ct.atomic_mass # ion mass
q = 1*ct.elementary_charge # ion charge
rf_scale = s.rf_scale(m,q,l,o)

s["RF"].rf = 25. # peak rf voltage
method = 'Newton-CG'
x0 = s.minimum((0, 0, 1.),method=method)
# , min_method=method
for _ in s.analyze_static(x0, m=m, l=l, o=o):
    print(_)

In [ ]:
n = 30
#xyz = np.mgrid[-.1:.1:1j*n, -.1:.1:1j*n, 1.12:2]
#xyz = np.mgrid[0:1, -.02:.02:1j*n, .5:1.5:1j*n]
xyz = grid.to_mgrid()
p = s.potential(xyz.reshape(3, -1).T, 0).reshape(xyz[0].shape)
v = np.linspace(0, 2000e-2, 21)
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.contour(xyz[1, 10, :, :], xyz[2, 10, :, :], p[10, :, :], v, cmap=plt.cm.Reds)